In [1]:

from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
import pandas as pd
import sklearn
from imblearn.combine import SMOTETomek
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,confusion_matrix,accuracy_score,classification_report
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.preprocessing import PolynomialFeatures,StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
pd.set_option('display.max_columns', 500)
import pickle as pkl

In [2]:
df = pd.read_csv('polydf.csv',index_col='Unnamed: 0')
df1 = pd.read_csv('prepeddata.csv',index_col='Unnamed: 0')
df1['stimulant'] = df1['stimulant'].apply(lambda x: 'low' if x <= 2 else 'high')
df
RandomForestClassifier()


RandomForestClassifier()

In [3]:
stimtestdata = df1.drop(['hallucinagen','depressant'],axis = 1)
stimx = df
stimy = df1.stimulant
stimx

sm = SMOTE()
tl = TomekLinks()
sampling = SMOTETomek(sampling_strategy='auto', random_state=None, smote=sm, tomek=tl, n_jobs=1)

stimx, stimy = sampling.fit_resample(stimx, stimy)



In [4]:
stimXtrain,stimXtest,stimYtrain,stimYtest = train_test_split(stimx,stimy)

In [5]:
stimscaler = StandardScaler()
stimXtrain = stimscaler.fit_transform(stimXtrain)
stimXtest = stimscaler.transform(stimXtest)
stimYtrain

1888    high
2986    high
210      low
350      low
2505    high
        ... 
1346     low
1908    high
457     high
3194    high
1520     low
Name: stimulant, Length: 2435, dtype: object

Logistic regression classifiers

In [6]:
# mod = LogisticRegression(class_weight = 'balanced'
#                          ,max_iter=1000,solver='liblinear',warm_start=True,tol=0.2)
# mod.fit(stimXtrain,stimYtrain)
# stimtrianlogypred = mod.predict(stimXtrain)
# stimtestlogypred = mod.predict(stimXtest)

In [7]:
logregparam_grid = {'C':[x/60 for x in list(range(0,60,1))],'max_iter' :[1000],'class_weight':['auto'],'warm_start' :[True,False], 'solver': ['liblinear', 'rbf','sag']}
mod = GridSearchCV(LogisticRegression(), logregparam_grid, n_jobs=-1, cv=5, verbose=1)
mod.fit(stimXtrain,stimYtrain)
stimtrianlogypred = mod.predict(stimXtrain)
stimtestlogypred = mod.predict(stimXtest)


Fitting 5 folds for each of 360 candidates, totalling 1800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 920 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 1800 out of 1800 | elapsed:   13.2s finished


In [8]:
polymodel1 = mod.best_estimator_


In [9]:
f1_score(stimYtrain,stimtrianlogypred,average='weighted'),f1_score(stimYtest,stimtestlogypred,average='weighted')

(0.735548722859226, 0.7529298095473556)

In [10]:
logregparam_grid2 = {'tol': [0,0.0001,0.001,0.1,1, 10]
                     ,'C': [0,0.001,0.1,1, 10]
                     ,'penalty': ['l1','l2','elasticnet']
                     ,'max_iter' :[1000],'class_weight':['auto']
                     ,'warm_start' :[True,False]
                     , 'solver': ['lbfgs','liblinear', 'rbf','sag','elasticnet','saga',]
                     ,'dual':[False,True],"l1_ratio":[None,0.01,0.1,0.2,0.3,0.4]
                    ,'fit_intercept':[True,False]}
mod2 = GridSearchCV(LogisticRegression(), logregparam_grid2, n_jobs=-1, cv=5, verbose=1)
mod2.fit(stimXtrain,stimYtrain)
stimtrianlog2ypred = mod2.predict(stimXtrain)
stimtestlog2ypred = mod2.predict(stimXtest)

Fitting 5 folds for each of 25920 candidates, totalling 129600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 3560 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 19560 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 32060 tasks      | elapsed:   30.4s
[Parallel(n_jobs=-1)]: Done 37650 tasks      | elapsed:   54.4s
[Parallel(n_jobs=-1)]: Done 55594 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 59959 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 68204 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 84156 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 90259 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 109557 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 114936 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 129600 out of 129600 | elapsed:  8.1min finished


In [11]:
f1_score(stimYtrain,stimtrianlog2ypred,average='weighted'),f1_score(stimYtest,stimtestlog2ypred,average='weighted')
polymodel2 = mod2.best_estimator_

In [12]:
logregparam_grid3 = {'tol': [0,0.0001,0.001,0.1,1, 10]
                     ,'penalty': ['l1','l2','elasticnet']
                     ,'max_iter' :[4000],'class_weight':['auto']
                     ,'warm_start' :[True]
                     , 'solver': ['lbfgs','liblinear', 'rbf','sag','elasticnet','saga']
                    }
mod3 = GridSearchCV(LogisticRegression(), logregparam_grid3, n_jobs=-1, cv=5, verbose=1)
mod3.fit(stimXtrain,stimYtrain)
stimtrianlog3ypred = mod3.predict(stimXtrain)
stimtestlog3ypred = mod3.predict(stimXtest)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:   17.4s finished


In [13]:
f1_score(stimYtrain,stimtrianlog3ypred,average='weighted'),f1_score(stimYtest,stimtestlog3ypred,average='weighted')

(0.7379043499630292, 0.7551881786251642)

In [14]:
polymodel3 = mod3.best_estimator_
#('polymodel1',polymodel1),('polymodel1',polymodel2),('polymodel1',polymodel3)

In [15]:
pkl.dump(polymodel1,open('PolyModel1.pkl','wb'))
pkl.dump(polymodel2,open('PolyModel2.pkl','wb'))
pkl.dump(polymodel3,open('PolyModel3.pkl','wb'))

Decision Trees

In [16]:
# dtc = DecisionTreeClassifier(criterion='gini',ccp_alpha=0.0055)
# dtc.fit(stimXtrain,stimYtrain)
# dtcytrainpred = dtc.predict(stimXtrain)
# dtcytestpred = dtc.predict(stimXtest)
parameters={'max_depth': range(1,20,1),'ccp_alpha':[x/60 for x in list(range(0,60,1))],'max_depth':[4,6,7,9,11],'criterion':['gini']}
dtc = GridSearchCV(DecisionTreeClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
dtc.fit(stimXtrain,stimYtrain)
dtcytrainpred = dtc.predict(stimXtrain)
dtcytestpred = dtc.predict(stimXtest)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 1000 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:    5.8s finished


In [17]:
f1_score(stimYtrain,dtcytrainpred,average='weighted'),f1_score(stimYtest,dtcytestpred,average='weighted')

(0.9506785233071624, 0.8347507642980759)

In [18]:
confusion_matrix(stimYtest,dtcytestpred)

array([[355,  53],
       [ 81, 323]])

In [19]:
parameters={'criterion':['entropy','gini'],'min_samples_leaf':range(0,20,2),'ccp_alpha':[x/300 for x in list(range(0,60,1))]}
dtc2 = GridSearchCV(DecisionTreeClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
dtc2.fit(stimXtrain,stimYtrain)
dtcytrainpred = dtc2.predict(stimXtrain)
dtcytestpred = dtc2.predict(stimXtest)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 1200 candidates, totalling 6000 fits


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 512 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 1512 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 2912 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done 4712 tasks      | elapsed:   23.9s
[Parallel(n_jobs=-1)]: Done 6000 out of 6000 | elapsed:   30.2s finished


In [20]:
f1_score(stimYtrain,dtcytrainpred,average='weighted'),f1_score(stimYtest,dtcytestpred,average='weighted')

(0.9843930396830458, 0.8496442201887768)

In [21]:
confusion_matrix(stimYtest,dtcytestpred)

array([[357,  51],
       [ 71, 333]])

In [22]:
# dtc3 = DecisionTreeClassifier(class_weight='balanced',criterion='gini',ccp_alpha=0.006599999)
# dtc3.fit(stimXtrain,stimYtrain)
# dtc3ytrainpred = dtc3.predict(stimXtrain)
# dtc3ytestpred = dtc3.predict(stimXtest)
parameters={'min_impurity_decrease':[0+x/100 for x in range(1,10)],'ccp_alpha':[x/60 for x in list(range(0,100,1))],'criterion':['entropy','gini'],'min_samples_leaf':range(0,20,2)}
dtc3 = GridSearchCV(DecisionTreeClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
dtc3.fit(stimXtrain,stimYtrain)
dtcytrainpred = dtc3.predict(stimXtrain)
dtcytestpred = dtc3.predict(stimXtest)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


Fitting 5 folds for each of 18000 candidates, totalling 90000 fits


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 1000 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 3000 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 5800 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done 9400 tasks      | elapsed:   19.4s
[Parallel(n_jobs=-1)]: Done 13800 tasks      | elapsed:   27.7s
[Parallel(n_jobs=-1)]: Done 19000 tasks      | elapsed:   38.3s
[Parallel(n_jobs=-1)]: Done 25000 tasks      | elapsed:   49.7s
[Parallel(n_jobs=-1)]: Done 31800 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 39400 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 47800 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 57000 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 67000 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 77800 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 89400 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 90000 out of 90000

In [23]:
f1_score(stimYtrain,dtcytrainpred,average='weighted'),f1_score(stimYtest,dtcytestpred,average='weighted')


(0.7642640816648272, 0.7609096616116616)

In [24]:
polymodel4 = dtc.best_estimator_
polymodel5 = dtc2.best_estimator_
polymodel6 = dtc3.best_estimator_

pkl.dump(polymodel4,open('PolyModel4.pkl','wb'))
pkl.dump(polymodel5,open('PolyModel5.pkl','wb'))
pkl.dump(polymodel6,open('PolyModel6.pkl','wb'))
#('polymodel1',polymodel1),('polymodel1',polymodel2),('polymodel1',polymodel3),('polymodel4',polymodel4),('polymodel5',polymodel5),('polymodel6',polymodel6)


Knearest

In [25]:
parameters={'n_neighbors':range(1,50)}
knn = GridSearchCV(KNeighborsClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
knn.fit(stimXtrain,stimYtrain)
knnytrainpred = knn.predict(stimXtrain)
knnytestpred = knn.predict(stimXtest)


Fitting 5 folds for each of 49 candidates, totalling 245 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 222 out of 245 | elapsed:    2.8s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done 245 out of 245 | elapsed:    2.9s finished


In [26]:
parameters={'n_neighbors':range(50,1130)}
knn2 = GridSearchCV(KNeighborsClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
knn2.fit(stimXtrain,stimYtrain)
knn2ytrainpred = knn2.predict(stimXtrain)
knn2ytestpred = knn2.predict(stimXtest)


Fitting 5 folds for each of 1080 candidates, totalling 5400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 328 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 828 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done 1528 tasks      | elapsed:   26.5s
[Parallel(n_jobs=-1)]: Done 2428 tasks      | elapsed:   44.8s
[Parallel(n_jobs=-1)]: Done 3528 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 4828 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 5400 out of 5400 | elapsed:  1.8min finished


In [27]:
polymodel7 = knn.best_estimator_
polymodel8 = knn2.best_estimator_

#('polymodel1',polymodel1),('polymodel1',polymodel2),('polymodel1',polymodel3),('polymodel4',polymodel4),('polymodel5',polymodel5),('polymodel6',polymodel6),('polymodel7',polymodel7),('polymodel8',polymodel8)
pkl.dump(polymodel7,open('PolyModel7.pkl','wb'))
pkl.dump(polymodel8,open('PolyModel8.pkl','wb'))


Random Forest

In [28]:
parameters={'warm_start':[True,False],'class_weight':['balanced'],'criterion':['gini','entropy'],'n_estimators':[20,50,100,500]}
rfc = GridSearchCV(RandomForestClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
rfc.fit(stimXtrain,stimYtrain)
rfcytrainpred = rfc.predict(stimXtrain)
rfcytestpred = rfc.predict(stimXtest)


Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   11.1s finished


In [29]:
rfctrainypred = rfc.predict(stimXtrain)
rfctestypred = rfc.predict(stimXtest)
f1_score(rfctrainypred,stimYtrain,average='weighted'),f1_score(rfctestypred,stimYtest,average='weighted')
confusion_matrix(stimYtest,rfctestypred)




array([[384,  24],
       [ 31, 373]])

In [30]:
parameters={'min_samples_leaf' : [3,5,7],'class_weight':['balanced'],'random_state':[True]
                         ,'criterion':['gini']
                         }
rfc2 = GridSearchCV(RandomForestClassifier(),param_grid=parameters,verbose=True,n_jobs=-1)
rfc2.fit(stimXtrain,stimYtrain)
rfc2ytrainpred = rfc2.predict(stimXtrain)
rfc2ytestpred = rfc2.predict(stimXtest)


Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:    0.7s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:    1.0s finished


In [31]:
rfc2trainypred = rfc2.predict(stimXtrain)
rfc2testypred = rfc2.predict(stimXtest)
f1_score(rfc2trainypred,stimYtrain,average='weighted'),f1_score(rfc2testypred,stimYtest,average='weighted')



(0.9880903450587022, 0.9175023302190882)

In [32]:
polymodel9 = rfc.best_estimator_
polymodel10 = rfc2.best_estimator_
#('polymodel1',polymodel1),('polymodel1',polymodel2),('polymodel1',polymodel3),('polymodel4',polymodel4),('polymodel5',polymodel5),('polymodel6',polymodel6),('polymodel7',polymodel7),('polymodel8',polymodel8),('polymodel9',polymodel9),('polymodel10',polymodel10)

pkl.dump(polymodel9,open('PolyModel9.pkl','wb'))
pkl.dump(polymodel10,open('PolyModel10.pkl','wb'))

voting classifier

In [58]:

vtc = VotingClassifier([('polymodel1',polymodel1),('polymodel2',polymodel2),('polymodel3',polymodel3)
                        ,('polymodel5',polymodel5),('polymodel6',polymodel6),('polymodel4',polymodel4)
                        ,('polymodel7',polymodel7),('polymodel8',polymodel8),('polymodel9',polymodel9)
                        ,('polymodel10',polymodel10)],voting='soft',verbose = True,n_jobs=-1)
vtc.fit(stimXtrain,stimYtrain)

VotingClassifier(estimators=[('polymodel1',
                              LogisticRegression(C=0.5333333333333333,
                                                 class_weight='auto',
                                                 max_iter=1000, solver='sag',
                                                 warm_start=True)),
                             ('polymodel2',
                              LogisticRegression(C=1, class_weight='auto',
                                                 fit_intercept=False,
                                                 max_iter=1000, tol=1,
                                                 warm_start=True)),
                             ('polymodel3',
                              LogisticRegression(class_weight='auto',
                                                 max_iter=4000, solver='sag',
                                                 tol=0.0...
                                                     min_impurity_decrease=0.01,
        

In [59]:
vtcypred = vtc.predict(stimXtest)
vtcytrain = vtc.predict(stimXtrain)

print(f1_score(vtcytrain,stimYtrain,average='weighted'),f1_score(stimYtest,vtcypred,average='weighted'))


0.9667425764880171 0.8783542307284208


In [60]:
pkl.dump(vtc,open('vtcpoly.pkl','wb'))

In [61]:
confusion_matrix(vtcytrain,stimYtrain)

array([[1196,   58],
       [  23, 1158]])

In [62]:
print(classification_report(stimYtrain, vtcytrain))

              precision    recall  f1-score   support

        high       0.95      0.98      0.97      1219
         low       0.98      0.95      0.97      1216

    accuracy                           0.97      2435
   macro avg       0.97      0.97      0.97      2435
weighted avg       0.97      0.97      0.97      2435



In [63]:
print(classification_report(stimYtest, vtcypred))

              precision    recall  f1-score   support

        high       0.82      0.97      0.89       408
         low       0.96      0.79      0.87       404

    accuracy                           0.88       812
   macro avg       0.89      0.88      0.88       812
weighted avg       0.89      0.88      0.88       812

